In [134]:
import pandas as pd
df = pd.read_csv("data/finalDataset.csv")
df.head()

,Unnamed: 0,Type,ProductBrand,Color,Size,LookBook,Gender,Fabric,ImageLink,Season,DiscountedPrice,Discount,ProductLink
0,0,nike men's relaxed fit t-shirt,nike,gold,S,casual,men,cotton,https://m.media-amazon.com/images/I/51UAabRZV2...,NaN,"2,489",47%,https://www.amazon.in/Nike-Relaxed-T-Shirt-DJ7...
1,1,puma men's regular fit t-shirt,puma,black white,S,sports,men,cotton,https://m.media-amazon.com/images/I/513rGsnZbo...,NaN,"1,439",20%,https://www.amazon.in/Puma-Printed-Regular-T-S...
2,2,nike women shirt,nike,black,M,sports,women,polyester,https://m.media-amazon.com/images/I/41h6g+dkZd...,NaN,809,55%,https://www.amazon.in/Nike-womens-Slim-Shirt-D...
3,3,puma men sweatshirt,puma,gray,XS,regular,men,cotton,https://m.media-amazon.com/images/I/61WRIRaAJp...,NaN,"1,499",50%,https://www.amazon.in/Puma-Cotton-Sweatshirt-8...
4,4,nike men casual shorts,nike,blue black,XS,casual,men,polyester,https://m.media-amazon.com/images/I/81NDwU5zk2...,NaN,"1,349",46%,https://www.amazon.in/Nike-Boxer-Shorts-DQ4813...


In [136]:
df = df.drop(columns=['Unnamed: 0',"Season" , "Size" , "ImageLink" , "ProductLink"])

In [137]:
df.columns

Index(['Type', 'ProductBrand', 'Color', 'LookBook', 'Gender', 'Fabric',
       'DiscountedPrice', 'Discount'],
      dtype='object')

In [138]:
df = df.dropna()

In [139]:
df.head()

,Type,ProductBrand,Color,LookBook,Gender,Fabric,DiscountedPrice,Discount
0,nike men's relaxed fit t-shirt,nike,gold,casual,men,cotton,"2,489",47%
1,puma men's regular fit t-shirt,puma,black white,sports,men,cotton,"1,439",20%
2,nike women shirt,nike,black,sports,women,polyester,809,55%
3,puma men sweatshirt,puma,gray,regular,men,cotton,"1,499",50%
4,nike men casual shorts,nike,blue black,casual,men,polyester,"1,349",46%


In [57]:
# df["DiscountedPrice"] = df["DiscountedPrice"].apply(lambda x : "price {} ".format(x))

In [140]:
df["str"] = " product name : " + df['Type'] +" company brand : "+ df["ProductBrand"] + " item color " + df["Color"] +  " style look : " + df["LookBook"] + " gender : " + df["Gender"] + " cloth fabric : " + df["Fabric"] + " discounted price : " + df["DiscountedPrice"] + " discount : " + df["Discount"]

In [141]:
from langchain_core.documents import Document
from langchain.text_splitter import CharacterTextSplitter
import os
from dotenv import load_dotenv
import requests

from langchain_google_genai import GoogleGenerativeAI
from langchain_community.embeddings import HuggingFaceInferenceAPIEmbeddings
import os
from dotenv import load_dotenv
from langchain import OpenAI
import openai
from langchain_community.vectorstores import Milvus
from langchain.chains import RetrievalQA 
from langchain.memory import ConversationBufferWindowMemory
from langchain import PromptTemplate
from langchain.memory import PostgresChatMessageHistory

load_dotenv()

True

In [142]:
for i in df["str"][:10] :
    print(i)

 product name : nike men's relaxed fit t-shirt company brand : nike item color gold style look : casual gender : men cloth fabric : cotton discounted price : 2,489 discount : 47%
 product name : puma men's regular fit t-shirt company brand : puma item color black white style look : sports gender : men cloth fabric : cotton discounted price : 1,439 discount : 20%
 product name : nike women shirt company brand : nike item color black style look : sports gender : women cloth fabric : polyester discounted price : 809 discount : 55%
 product name : puma men sweatshirt company brand : puma item color gray style look : regular gender : men cloth fabric : cotton discounted price : 1,499 discount : 50%
 product name : nike men casual shorts company brand : nike item color blue black style look : casual gender : men cloth fabric : polyester discounted price : 1,349 discount : 46%
 product name : adidas men's regular track pants company brand : adidas item color black white style look : birthday 

In [143]:
seperator = "\n ************ \n"
data = " "
for i in df["str"]:
    # print(type(data) ,type(i) ,type(seperator) , )
    data = data + i + seperator


In [144]:
def get_docs(collection_name):
    # data = get_data(collection_name)
    text_splitter = CharacterTextSplitter(
        separator = "************",
        chunk_size=400,
        chunk_overlap=0
    )
    dataDocument = [Document(page_content=data, metadata={"namespace": collection_name})]
    docs = text_splitter.split_documents(dataDocument)
    return docs



def create_vectorDB(collection_name):
    docs = get_docs(collection_name)
    initialize_embeddings()
    global vector_db
    vector_db = Milvus.from_documents(
        docs,
        embeddings,
        collection_name=collection_name,
        connection_args={
            "uri":os.getenv("ZILLIZ_CLOUD_URI") ,
            "user":os.getenv("ZILLIZ_CLOUD_USERNAME") ,
            "password":os.getenv("ZILLIZ_CLOUD_PASSWORD") ,
            # "token": ZILLIZ_CLOUD_API_KEY,  # API key, for serverless clusters which can be used as replacements for user and password
            "secure": True,
        },
    )


def initialize_embeddings():
    try :
        global embeddings
        embeddings = HuggingFaceInferenceAPIEmbeddings(
            api_key=os.getenv("HUGGINGFACE_API_KEY"), model_name="sentence-transformers/all-MiniLM-l6-v2"
        )
    except Exception as e :
        print("probelm accured while creating embeddings \n{}".format(e))


In [145]:
# create_vectorDB("similarity")
initialize_embeddings()

In [50]:
# text_splitter = CharacterTextSplitter(
#     separator = "************",
#     chunk_size=400,
#     chunk_overlap=0
# )
# dataDocument = [Document(page_content=data, metadata={"namespace": "simm"})]
# docs = text_splitter.split_documents(dataDocument)

In [147]:
docs

[Document(page_content="product name : Nike Men's Relaxed Fit T-Shirt company brand : Nike item color Gold item size : S style look : Casual gender : men cloth fabric : Cotton discounted price : price 2,489  discount : 47%", metadata={'namespace': 'simm'}),
 Document(page_content="product name : Nike Men's Relaxed Fit T-Shirt company brand : Nike item color Gold item size : M style look : Casual gender : men cloth fabric : Cotton discounted price : price 2,489  discount : 47%", metadata={'namespace': 'simm'}),
 Document(page_content="product name : Nike Men's Relaxed Fit T-Shirt company brand : Nike item color Gold item size : L style look : Casual gender : men cloth fabric : Cotton discounted price : price 2,489  discount : 47%", metadata={'namespace': 'simm'}),
 Document(page_content="product name : Nike Men's Relaxed Fit T-Shirt company brand : Nike item color Gold item size : XL style look : Casual gender : men cloth fabric : Cotton discounted price : price 2,489  discount : 47%", 

In [148]:
def load_vectorDB(collection_name):
    global vector_db
    vector_db = Milvus(
    embeddings,
    collection_name=collection_name,
    connection_args={
    "uri":os.getenv("ZILLIZ_CLOUD_URI") ,
    "user":os.getenv("ZILLIZ_CLOUD_USERNAME") ,
    "password":os.getenv("ZILLIZ_CLOUD_PASSWORD") ,
    # "token": ZILLIZ_CLOUD_API_KEY,  # API key, for serverless clusters which can be used as replacements for user and password
    "secure": True,
    },
)
    
load_vectorDB("similarity")


In [24]:
print(data)

  product name : Nike Men's Relaxed Fit T-Shirt company brand : Nike item color Gold item size : S style look : Casual gender : men cloth fabric : Cotton discounted price : price 2,489  discount : 47%
 ************ 
 product name : Nike Men's Relaxed Fit T-Shirt company brand : Nike item color Gold item size : M style look : Casual gender : men cloth fabric : Cotton discounted price : price 2,489  discount : 47%
 ************ 
 product name : Nike Men's Relaxed Fit T-Shirt company brand : Nike item color Gold item size : L style look : Casual gender : men cloth fabric : Cotton discounted price : price 2,489  discount : 47%
 ************ 
 product name : Nike Men's Relaxed Fit T-Shirt company brand : Nike item color Gold item size : XL style look : Casual gender : men cloth fabric : Cotton discounted price : price 2,489  discount : 47%
 ************ 
 product name : Puma Men's Regular Fit T-Shirt company brand : Puma item color Black/White item size : S style look : Sports gender : men 

In [149]:
vector_db.similarity_search(query="#casual #black #nike" , k=10)

[Document(page_content="product name : nike men casual shorts company brand : nike item color blue black style look : casual gender : men cloth fabric : polyester discounted price : 1,349 discount : 46%\n ************ \n product name : adidas men's regular track pants company brand : adidas item color black white style look : birthday regular gender : men cloth fabric : cotton discounted price : 1,799 discount : 40%", metadata={'namespace': 'similarity'}),
 Document(page_content="product name : nike men's relaxed fit t-shirt company brand : nike item color gold style look : casual gender : men cloth fabric : cotton discounted price : 2,489 discount : 47%\n ************ \n product name : puma men's regular fit t-shirt company brand : puma item color black white style look : sports gender : men cloth fabric : cotton discounted price : 1,439 discount : 20%", metadata={'namespace': 'similarity'}),
 Document(page_content='product name : nike women shirt company brand : nike item color black

In [150]:
llm = GoogleGenerativeAI(model="gemini-pro", google_api_key=os.getenv("GOOGLE_API_KEY") , temperature=0.6)


In [177]:

from langchain_core.messages import HumanMessage, SystemMessage
from langchain_google_genai import ChatGoogleGenerativeAI


model = ChatGoogleGenerativeAI(model="gemini-pro", convert_system_message_to_human=True)
query = "how are you"
tags = model(
    [
        SystemMessage(content="you are a tag generator. you have to assign tags to user query which can be used to suggest products to the customer .do not type any other things only provide tags. tags could be any color , size , gender ,cloth fabric in [Cotton polyester Silk Blend Acrylic Wool Jean Polyester Faux Georgette Silk Jacquard] or design catagory in [Casual Sports Regular Birthday Formal Clubwear Partywear Funky Sweater casual Aesthetic Sport traditional]. if you do not find any relavant tags then return None."),
        HumanMessage(content=query),
    ]
)

In [178]:
print(tags.content)


None


In [162]:
simtag = vector_db.similarity_search(query=tags.content , k=10)

In [163]:
simtag

[Document(page_content='product name : ellipse jackets for women full sleeves winter jackets with inner mobile pocket jackets with zip closure company brand : ellipse item color green style look : winter gender : women cloth fabric : nylon discounted price : 2,160 discount : 57%', metadata={'namespace': 'similarity'}),
 Document(page_content="product name : abbl fashionable full sleeves winter men's standard length polyester jacket for stylish comfortable huddy for adult boys company brand : abbl item color black grey style look : winter gender : men cloth fabric : polyester discounted price : 799 discount : 20%", metadata={'namespace': 'similarity'}),
 Document(page_content='product name : leotude oversized cottonblend half sleeve back printed drop shoulder men t-shirt company brand : leotude item color black blue style look : funky gender : men cloth fabric : cotton blend discounted price : 289 discount : 74%', metadata={'namespace': 'similarity'}),
 Document(page_content='product na

In [164]:
simq = vector_db.similarity_search(query=query , k=10)

In [165]:
simq

[Document(page_content="product name : abbl fashionable full sleeves winter men's standard length polyester jacket for stylish comfortable huddy for adult boys company brand : abbl item color black grey style look : winter gender : men cloth fabric : polyester discounted price : 799 discount : 20%", metadata={'namespace': 'similarity'}),
 Document(page_content='product name : ellipse jackets for women full sleeves winter jackets with inner mobile pocket jackets with zip closure company brand : ellipse item color green style look : winter gender : women cloth fabric : nylon discounted price : 2,160 discount : 57%', metadata={'namespace': 'similarity'}),
 Document(page_content='product name : pipasa women woolen buttoned collar neck stylish party winter wear coat cardigan sweater company brand : pipasa item color brown grey pista red style look : sweater winter gender : women cloth fabric : wool discounted price : 899 discount : 55%', metadata={'namespace': 'similarity'}),
 Document(page

In [166]:
contextt = simtag[0].page_content + " --- " + simtag[1].page_content + " --- " + simtag[2].page_content + " --- " + simq[0].page_content + " --- " + simq[1].page_content + " --- " + simq[2].page_content

In [167]:
contextt

"product name : ellipse jackets for women full sleeves winter jackets with inner mobile pocket jackets with zip closure company brand : ellipse item color green style look : winter gender : women cloth fabric : nylon discounted price : 2,160 discount : 57% --- product name : abbl fashionable full sleeves winter men's standard length polyester jacket for stylish comfortable huddy for adult boys company brand : abbl item color black grey style look : winter gender : men cloth fabric : polyester discounted price : 799 discount : 20% --- product name : leotude oversized cottonblend half sleeve back printed drop shoulder men t-shirt company brand : leotude item color black blue style look : funky gender : men cloth fabric : cotton blend discounted price : 289 discount : 74% --- product name : abbl fashionable full sleeves winter men's standard length polyester jacket for stylish comfortable huddy for adult boys company brand : abbl item color black grey style look : winter gender : men clot

In [168]:
llm.invoke("""you are a salse person. try to give human like response according to user query and given context.
    Use the following context (delimited by <ctx></ctx>) and the chat history (delimited by <hs></hs>) to answer the question:
    ------
    <ctx>
    {}
    </ctx>
    ------
    <hs> </hs>
    {}
    Answer:""".format(contextt,query ))

"I understand you're feeling cold. May I suggest taking a look at our warm and stylish winter jackets? We have a variety of options to choose from, so you're sure to find one that's perfect for you."

In [170]:
llm.invoke("""you are a salse person. try to give human like response according to user query and given context.
    Use the following context (delimited by <ctx></ctx>) and the chat history (delimited by <hs></hs>) to answer the question:
    ------
    <ctx>
    {}
    </ctx>
    ------
    <hs>
    bot : "I understand you're feeling cold. May I suggest taking a look at our warm and stylish winter jackets? We have a variety of options to choose from, so you're sure to find one that's perfect for you."
    user : "yes sure"
    bot : "Absolutely! Let me show you our collection of winter jackets. We have a wide range of styles and colors to choose from, so you're sure to find the perfect one for your needs."
    </hs>
    {}
    Answer:""".format(contextt,"show me the best" ))

"Of course! Our most popular winter jacket is the ellipse jackets for women full sleeves winter jackets with inner mobile pocket jackets with zip closure. It's made from high-quality nylon and features a stylish design. It's also water-resistant and windproof, so you'll stay warm and dry even in the coldest weather."